# Capstone Project: Supply Chain Monitoring and Optimization Platform

# Week 4 – Simple ETL in Azure Databricks

Tools: Azure Databricks

Capstone Tasks:

    1. Upload CSV data into Databricks
    2. Run a notebook to clean and filter the data
    3. Save cleaned output as Delta or CSV
    4. Run basic analysis queries using SQL or PySpark

In [1]:
# Install PySpark
!pip install pyspark -q

In [2]:
# Import the Requirements
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, current_date, datediff

In [3]:
# Install dependencies (only needed in Google Colab)
# In Azure Databricks, skip this cell
!pip install pyspark==3.5.1 delta-spark==3.1.0
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

In [4]:
# using DELTA in colab
!pip install delta-spark==3.2.0 -q
import pyspark
from delta import *
from pyspark.sql.functions import *

# Create a SparkSession with Delta Lake extensions
# The '.config(...)' lines are crucial for enabling Delta Lake's features
builder = pyspark.sql.SparkSession.builder.appName("DeltaDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Get or create the SparkSession
spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("Spark and Delta Lake are ready!")

Spark and Delta Lake are ready!


In [5]:
# Upload your CSV files manually (orders.csv, suppliers.csv, inventory.csv)
from google.colab import files

print("Upload orders.csv")
orders_upload = files.upload()

print("Upload suppliers.csv")
suppliers_upload = files.upload()

print("Upload inventory.csv")
inventory_upload = files.upload()

Upload orders.csv


Saving orders.csv to orders.csv
Upload suppliers.csv


Saving suppliers.csv to suppliers.csv
Upload inventory.csv


Saving inventory.csv to inventory.csv


In [9]:
# 1. Load CSV files into Spark DataFrames
orders_df = spark.read.csv("orders.csv", header=True, inferSchema=True)
suppliers_df = spark.read.csv("suppliers.csv", header=True, inferSchema=True)
inventory_df = spark.read.csv("inventory.csv", header=True, inferSchema=True)

print("Orders Data:")
orders_df.show()

Orders Data:
+--------+----------+-----------+--------+----------+-------------+---------+
|order_id|product_id|supplier_id|quantity|order_Date|delivery_date|   status|
+--------+----------+-----------+--------+----------+-------------+---------+
|       1|         3|          1|      40|2025-07-03|   2025-07-07|  Shipped|
|       2|         7|          4|      15|2025-07-06|   2025-07-11|Delivered|
|       3|         2|          2|      25|2025-07-01|   2025-07-06|  Pending|
|       4|        10|          5|      10|2025-07-10|   2025-07-14|Cancelled|
|       5|         1|          3|      50|2025-07-02|   2025-07-07|Delivered|
|       6|         8|          8|      20|2025-07-07|   2025-07-12|  Pending|
|       7|         5|          6|      35|2025-07-05|   2025-07-09|Delivered|
|       8|         9|         10|      60|2025-07-08|   2025-07-13|  Shipped|
|       9|         6|          7|     100|2025-07-04|   2025-07-08|Delivered|
|      10|         4|          9|      12|2025-07-0

In [10]:
# 2. Clean & Transform Orders Data
orders_df_clean = orders_df \
    .withColumn("order_date", to_date(col("order_date"), "yyyy-MM-dd")) \
    .withColumn("delivery_date", to_date(col("delivery_date"), "yyyy-MM-dd")) \
    .withColumn("delay_days", datediff(current_date(), col("delivery_date"))) \
    .withColumn("is_delayed", (col("delay_days") > 0).cast("int"))

print("Cleaned Orders Data:")
orders_df_clean.show()

Cleaned Orders Data:
+--------+----------+-----------+--------+----------+-------------+---------+----------+----------+
|order_id|product_id|supplier_id|quantity|order_date|delivery_date|   status|delay_days|is_delayed|
+--------+----------+-----------+--------+----------+-------------+---------+----------+----------+
|       1|         3|          1|      40|2025-07-03|   2025-07-07|  Shipped|        39|         1|
|       2|         7|          4|      15|2025-07-06|   2025-07-11|Delivered|        35|         1|
|       3|         2|          2|      25|2025-07-01|   2025-07-06|  Pending|        40|         1|
|       4|        10|          5|      10|2025-07-10|   2025-07-14|Cancelled|        32|         1|
|       5|         1|          3|      50|2025-07-02|   2025-07-07|Delivered|        39|         1|
|       6|         8|          8|      20|2025-07-07|   2025-07-12|  Pending|        34|         1|
|       7|         5|          6|      35|2025-07-05|   2025-07-09|Delivered|  

In [11]:
# Filter delayed orders
delayed_orders_df = orders_df_clean.filter(col("is_delayed") == 1)
display(delayed_orders_df)

DataFrame[order_id: int, product_id: int, supplier_id: int, quantity: int, order_date: date, delivery_date: date, status: string, delay_days: int, is_delayed: int]

In [12]:
# 3. Save cleaned output as Delta or CSV
orders_df_clean.write.format("delta").mode("overwrite").save("/FileStore/tables/orders_cleaned_delta")
orders_df_clean.write.format("csv").option("header", "true").mode("overwrite").save("/FileStore/tables/orders_cleaned_csv")

In [13]:
# 4. Basic Analysis in PySpark

# 4.1 Count delayed orders by supplier_id
delays_by_supplier = delayed_orders_df.groupBy("supplier_id").count()
print("Delay count per supplier:")
delays_by_supplier.show()

Delay count per supplier:
+-----------+-----+
|supplier_id|count|
+-----------+-----+
|          1|    1|
|          6|    1|
|          3|    1|
|          5|    1|
|          9|    1|
|          4|    1|
|          8|    1|
|          7|    1|
|         10|    1|
|          2|    1|
+-----------+-----+



In [14]:
# 4.2 Average delay days per supplier
avg_delay_by_supplier = delayed_orders_df.groupBy("supplier_id").agg(avg("delay_days").alias("avg_delay_days"))
print("Average delay days per supplier:")
avg_delay_by_supplier.show()

Average delay days per supplier:
+-----------+--------------+
|supplier_id|avg_delay_days|
+-----------+--------------+
|          1|          39.0|
|          6|          37.0|
|          3|          39.0|
|          5|          32.0|
|          9|          31.0|
|          4|          35.0|
|          8|          34.0|
|          7|          38.0|
|         10|          33.0|
|          2|          40.0|
+-----------+--------------+



In [15]:
# 4.3 Maximum and minimum delay days
max_min_delay = delayed_orders_df.agg(max("delay_days").alias("max_delay"), min("delay_days").alias("min_delay"))
print("Maximum and minimum delay days:")
max_min_delay.show()

Maximum and minimum delay days:
+---------+---------+
|max_delay|min_delay|
+---------+---------+
|       40|       31|
+---------+---------+



In [16]:
# 4.4 Orders count by status
orders_by_status = orders_df_clean.groupBy("status").count()
print("Order count by status:")
orders_by_status.show()

Order count by status:
+---------+-----+
|   status|count|
+---------+-----+
|  Shipped|    2|
|Cancelled|    1|
|Delivered|    5|
|  Pending|    2|
+---------+-----+



In [24]:
# 5. Basic Analysis in SQL

orders_df_clean.createOrReplaceTempView("orders_cleaned")

# 5.1 Suppliers with more than 1 delayed order
spark.sql("""
SELECT supplier_id, COUNT(*) AS delayed_count
FROM orders_cleaned
WHERE is_delayed = 1
GROUP BY supplier_id
HAVING delayed_count > 1
""").show()

+-----------+-------------+
|supplier_id|delayed_count|
+-----------+-------------+
+-----------+-------------+



In [25]:
# 5.2 Average delay per product
spark.sql("""
SELECT product_id, AVG(delay_days) AS avg_delay
FROM orders_cleaned
WHERE is_delayed = 1
GROUP BY product_id
ORDER BY avg_delay DESC
""").show()

+----------+---------+
|product_id|avg_delay|
+----------+---------+
|         2|     40.0|
|         1|     39.0|
|         3|     39.0|
|         6|     38.0|
|         5|     37.0|
|         7|     35.0|
|         8|     34.0|
|         9|     33.0|
|        10|     32.0|
|         4|     31.0|
+----------+---------+



In [26]:
# 5.3 Orders delivered
spark.sql("""
SELECT *
FROM orders_cleaned
WHERE status = 'Delivered'
""").show()

+--------+----------+-----------+--------+----------+-------------+---------+----------+----------+
|order_id|product_id|supplier_id|quantity|order_date|delivery_date|   status|delay_days|is_delayed|
+--------+----------+-----------+--------+----------+-------------+---------+----------+----------+
|       2|         7|          4|      15|2025-07-06|   2025-07-11|Delivered|        35|         1|
|       5|         1|          3|      50|2025-07-02|   2025-07-07|Delivered|        39|         1|
|       7|         5|          6|      35|2025-07-05|   2025-07-09|Delivered|        37|         1|
|       9|         6|          7|     100|2025-07-04|   2025-07-08|Delivered|        38|         1|
|      10|         4|          9|      12|2025-07-09|   2025-07-15|Delivered|        31|         1|
+--------+----------+-----------+--------+----------+-------------+---------+----------+----------+



Deliverables:

    1. Azure Databricks notebook
    2. Cleaned output stored in Delta/CSV format

In [27]:
# Save cleaned output as a single CSV file
clean_csv_path = "/content/orders_cleaned.csv"

In [28]:
# Coalesce to 1 partition so Spark writes only one CSV file
orders_df_clean.coalesce(1).write.option("header", "true").mode("overwrite").csv("/content/orders_cleaned_single")

In [29]:
# Rename part file to a nice CSV name
import shutil, glob
part_file = glob.glob("/content/orders_cleaned_single/part-*.csv")[0]
shutil.move(part_file, clean_csv_path)

print(f"Cleaned CSV saved at: {clean_csv_path}")

Cleaned CSV saved at: /content/orders_cleaned.csv


In [30]:
# Download directly in Colab
from google.colab import files
files.download(clean_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>